In [1]:
import numpy as np
import gensim
import string
import re
import math


from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation, SimpleRNN
from keras.models import Sequential
from keras.utils.data_utils import get_file


from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#PreProcessing

In [3]:
!ls
%cd drive/
!ls
%cd "My Drive"
# !ls
%cd assigmment_nlp
!ls


max_length = 40
with open("speeches.txt", 'r', encoding='utf-8') as myfile:
  c = myfile.read()
  c=c.lower()
  token = sent_tokenize(c)
  token = [re.sub(r'[^a-z0-9]'," ", i) for i in token]
  
sent_token = ["<s> " + i + "</s>" for i in token]
# token

sentences = [sent.split()[:max_length] for sent in sent_token]

train = sentences[:int(len(sentences)*0.8)]
test = sentences[int(len(sentences)*0.8):]

# print(test)

sentences = train


word_model = gensim.models.Word2Vec(sentences, size=100, min_count=1, window=5, iter=100)


drive  sample_data
/content/drive
'My Drive'  'Shared drives'
/content/drive/My Drive
/content/drive/My Drive/assigmment_nlp
austin.txt  speeches.txt


#Getting word vectors from word2vec

In [4]:
trained_weights = word_model.wv.vectors
trained_vocabs = list(word_model.wv.vocab)
# word_model["</s>"]
vocab_size = word_model.wv.vectors.shape[0]
embedding_size = word_model.wv.vectors.shape[1]
# print(embedding_size)

print(vocab_size)

def word2idx(word):
  return word_model.wv.vocab[word].index       #https://github.com/carpedm20/word2vec-tensorflow/blob/master/model.py 
def idx2word(idx):                             #reference 
  return word_model.wv.index2word[idx]



# print(trained_weights[0])

# print(word2idx('thank'))

# word_model.wv.vocab['</s>'].index

word_model.wv.index2word[122]

5177


'things'

#Model Initialization LSTM

In [5]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[trained_weights]))
lstm_model.add(LSTM(units=embedding_size))
lstm_model.add(Dense(units=vocab_size))
lstm_model.add(Activation('softmax'))
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

#Getting Input and Output Samples for Model

In [6]:
print(len(sentences))
train_x = np.zeros([len(sentences), max_length], dtype=np.int32)
# print(train_x)
train_y = np.zeros([len(sentences)], dtype=np.int32)
print(train_y.shape)
for i, sentence in enumerate(sentences):
  for t, word in enumerate(sentence[:-1]):
    train_x[i, t] = word2idx(word)
  train_y[i] = word2idx(sentence[-1])
# print('train_x shape:', train_x.shape)
# print('train_y shape:', train_y.shape)

13120
(13120,)


#Training the Model LSTM

In [7]:
lstm_model.fit(train_x, train_y,
          batch_size=128,
          epochs=20)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/20
13120/13120 [==============================] - 8s 614us/step - loss: 1.8626
Epoch 2/20
13120/13120 [==============================] - 7s 549us/step - loss: 0.1275
Epoch 3/20
13120/13120 [==============================] - 7s 543us/step - loss: 0.1069
Epoch 4/20
13120/13120 [==============================] - 7s 545us/step - loss: 0.0934
Epoch 5/20
13120/13120 [==============================] - 7s 545us/step - loss: 0.0837
Epoch 6/20
13120/13120 [==============================] - 7s 545us/step - loss: 0.0760
Epoch 7/20
13120/13120 [==============================] - 7s 545us/step - loss: 0.0683
Epoch 8/20
13120/13120 [==============================] - 7s 553us/step - loss: 0.0616
Epoch 9/20
13120/13120 [==============================] - 7s 549us/step - loss: 0.0545
Epoch 10/20
13120/13120 [==============================] - 7s 545us/step - loss: 0.0477
Epoch 11/20
13120/13120 [==========

#Generating Sentences from LSTM Model

In [8]:
start_arr = ['They are','so','here','we','with']
print("Sentences generated by LSTM : \n")
for i in start_arr:

  
  len_of_seq = 3
  word_idxs = [word2idx(word) for word in i.lower().split()]



  for i in range(len_of_seq):
    preds = lstm_model.predict(x=np.array([word_idxs]))

    probas = np.random.multinomial(1, preds[0], 1)

    idx = np.argmax(probas)

    word_idxs.append(idx)

  print(' '.join(idx2word(idx) for idx in word_idxs))


Sentences generated by LSTM : 

they are interests shooters vibrant
so wife of than
here stupidly lucky each
we colonel arrested 53
with hasn sign their


#Model for RNN

In [0]:
rnn_model = Sequential()
rnn_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, weights=[trained_weights]))
rnn_model.add(SimpleRNN(units=embedding_size))
rnn_model.add(Dense(units=vocab_size))
rnn_model.add(Activation('softmax'))
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

#Training RNN Model

In [10]:
rnn_model.fit(train_x, train_y,
          batch_size=128,
          epochs=20)

Epoch 1/20
13120/13120 [==============================] - 2s 187us/step - loss: 1.9799
Epoch 2/20
13120/13120 [==============================] - 2s 157us/step - loss: 0.1560
Epoch 3/20
13120/13120 [==============================] - 2s 156us/step - loss: 0.1218
Epoch 4/20
13120/13120 [==============================] - 2s 156us/step - loss: 0.1032
Epoch 5/20
13120/13120 [==============================] - 2s 154us/step - loss: 0.0914
Epoch 6/20
13120/13120 [==============================] - 2s 156us/step - loss: 0.0830
Epoch 7/20
13120/13120 [==============================] - 2s 156us/step - loss: 0.0760
Epoch 8/20
13120/13120 [==============================] - 2s 155us/step - loss: 0.0699
Epoch 9/20
13120/13120 [==============================] - 2s 154us/step - loss: 0.0640
Epoch 10/20
13120/13120 [==============================] - 2s 155us/step - loss: 0.0581
Epoch 11/20
13120/13120 [==============================] - 2s 154us/step - loss: 0.0522
Epoch 12/20
13120/13120 [================

#Generating Sentences from RNN Model

In [11]:
start_arr = ['They are','so','here','we','with']
print("Sentences generated by RNN : \n")
for i in start_arr:

  
  len_of_seq = 3
  word_idxs = [word2idx(word) for word in i.lower().split()]



  for i in range(len_of_seq):
    preds = rnn_model.predict(x=np.array([word_idxs]))

    probas = np.random.multinomial(1, preds[0], 1)

    idx = np.argmax(probas)

    word_idxs.append(idx)

  print(' '.join(idx2word(idx) for idx in word_idxs))

Sentences generated by RNN : 

they are walker dudes announcers
so having ho pages
here steel jewish columbus
we strengthen arrange door
with of he accuse


#Perplexity of LSTM Model


In [12]:

sent_data=test

word_model = gensim.models.Word2Vec(sent_data, size=100, min_count=1, iter=100)
trained_vocabs = word_model.wv.vocab
trained_weights = np.array(word_model.wv.vectors)
vocab_size = word_model.wv.vectors.shape[0]
embedding_size = word_model.wv.vectors.shape[1]

      

perplexity=0
for i, line in enumerate(sent_data):
  x=line[:-1]
  y=line[-1]
  windex_x=[word_model.wv.vocab[word].index for word in x]
  preds=lstm_model.predict(x=np.array([windex_x]))
  idx=word_model.wv.vocab[y].index
  prob=preds[0][idx]
  perplexity=perplexity-(math.log(prob)/len(sent_data))
perplexity=math.exp(perplexity)

perplexity

30.403669349067883

###Readability of LSTM and RNN models is much better than n-gram models

### LSTM is a better than RNN in terms of Perplexity ### 